# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Blackmans Bay,-43.0167,147.3167,12.07,78,100,3.02,AU,1684791736
1,1,Turabah,21.2141,41.6331,27.58,33,32,1.72,SA,1684791983
2,2,Qaqortoq,60.7167,-46.0333,-0.47,97,100,10.70,GL,1684791984
3,3,Nagqu Prefecture,31.5000,92.0000,-1.96,34,14,1.33,CN,1684791856
4,4,Hermanus,-34.4187,19.2345,14.33,61,100,2.98,ZA,1684791984


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
#%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1,
    color="City",
    alpha = 0.7
)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_spots_df = city_data_df.loc[
    (city_data_df["Max Temp"]<25) & (city_data_df["Max Temp"] > 16)
    & (city_data_df["Humidity"]<70)
    & (city_data_df["Wind Speed"]>5)
    & (city_data_df["Cloudiness"]<25)
]

# Drop any rows with null values
vacation_spots_df.isnull().sum()

# Display sample data
vacation_spots_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,Shelton,41.3165,-73.0932,21.91,63,0,5.14,US,1684792000
158,158,Douz,33.4663,9.0203,22.76,46,10,9.24,TN,1684792009
175,175,Hudson,40.7168,-74.0663,24.14,58,0,7.20,US,1684791908
323,323,Pisco,-13.7000,-76.2167,24.03,64,0,5.66,PE,1684792014
342,342,Jamestown,42.0970,-79.2353,22.51,44,0,5.14,US,1684791850
393,393,Ketchikan,55.3422,-131.6461,16.26,39,0,5.66,US,1684792059
558,558,Şurmān,32.7567,12.5716,20.30,66,0,5.61,LY,1684792080


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_spots_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy(deep=False)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
79,Shelton,US,41.3165,-73.0932,63,
158,Douz,TN,33.4663,9.0203,46,
175,Hudson,US,40.7168,-74.0663,58,
323,Pisco,PE,-13.7000,-76.2167,64,
342,Jamestown,US,42.0970,-79.2353,44,
393,Ketchikan,US,55.3422,-131.6461,39,
558,Şurmān,LY,32.7567,12.5716,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Shelton - nearest hotel: Hilton Garden Inn
Douz - nearest hotel: Louage station Douz
Hudson - nearest hotel: Ramada by Wyndham Jersey City
Pisco - nearest hotel: La Portada
Jamestown - nearest hotel: DoubleTree Jamestown
Ketchikan - nearest hotel: Cape Fox Lodge
Şurmān - nearest hotel: فندق البرج


,City,Country,Lat,Lng,Humidity,Hotel Name
79,Shelton,US,41.3165,-73.0932,63,Hilton Garden Inn
158,Douz,TN,33.4663,9.0203,46,Louage station Douz
175,Hudson,US,40.7168,-74.0663,58,Ramada by Wyndham Jersey City
323,Pisco,PE,-13.7000,-76.2167,64,La Portada
342,Jamestown,US,42.0970,-79.2353,44,DoubleTree Jamestown
393,Ketchikan,US,55.3422,-131.6461,39,Cape Fox Lodge
558,Şurmān,LY,32.7567,12.5716,66,فندق البرج


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:


# Configure the map plot
hotels_map = hotel_df.hvplot.points( 
    "Lng",
    "Lat",
    geo = True,
    tiles= "OSM",
    frame_width = 800,
    frame_height = 600,
     size = "Humidity",
    scale = 1.7,
    color= "City",
    hover_cols = ["Hotel Name", "Country"]

)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)